### 1. 引入大模型

In [1]:
from utils import get_ernie_models
from utils import get_qwen_models

In [2]:
llm_qwen, chat_qwen, embed_qwen = get_qwen_models()
llm_ernie, chat_ernie, embed_ernie = get_ernie_models()

### 2. 引入一段文本

In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
text_loader = TextLoader(file_path="./大语言模型.txt", encoding="utf8")

In [5]:
docs = text_loader.load()

### 3. 切分文本

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
spliter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=64)

In [8]:
docs = spliter.split_documents(documents=docs)

### 4. 向量化

In [9]:
# langchain 操控 Chroma
from langchain_chroma import Chroma

# Chroma 原生操作
import chromadb
from chromadb import Client
from chromadb import Settings

In [10]:
# 配置连接信息
setting = Settings(chroma_server_host="localhost", 
                  chroma_server_http_port=8001)

In [12]:
client = Client(settings=setting)

In [18]:
store = Chroma(collection_name="langchain", 
               embedding_function=embed_ernie,
               client=client)

In [20]:
store.add_documents(documents=docs)

[INFO][2024-08-24 16:37:09.388] oauth.py:228 [t:16180]: trying to refresh access_token for ak `66Cbfz***`
[INFO][2024-08-24 16:37:09.470] oauth.py:243 [t:16180]: sucessfully refresh access_token


['9f841db6-6a62-4a97-9e3e-415df6dd54c8',
 'f851730b-6adf-4279-b359-0e64a8f556f3',
 'c2e8d9b4-18cc-4b20-9356-b8ce60d66f1d',
 'd77e66c6-c171-4fd8-9480-352668bebce7',
 'e4cbaf6b-5e65-49f9-aaed-7612051e027e',
 '3d45b42f-69ac-4616-bc59-83d52355ec54',
 'ec9f0499-8320-4760-83ef-31e3acc4c185',
 'f739b44e-b2ac-4114-a47d-dc2e071c4e18',
 '8a3aa050-3f4b-4158-a9e1-f9ada4ed9b6b',
 '4310f7e7-a62e-4b3b-9f9c-952afb11c643',
 '42b75999-ab9e-4e95-9205-206f12e2da3a',
 '36f98cb4-9416-43ac-8512-34ac5613fe68',
 '805868b5-fcce-430e-a66b-55b37389aa87']

In [22]:
store.get()

{'ids': ['36f98cb4-9416-43ac-8512-34ac5613fe68',
  '3d45b42f-69ac-4616-bc59-83d52355ec54',
  '42b75999-ab9e-4e95-9205-206f12e2da3a'],
 'embeddings': None,
 'metadatas': [{'source': './大语言模型.txt'},
  {'source': './大语言模型.txt'},
  {'source': './大语言模型.txt'}],
 'documents': ['10. 语音识别与生成\n描述：大语言模型在语音识别和语音生成方面也展现出巨大潜力。通过将语音转录为文本或将文本转化为语音，该技术使得人们与计算机的交互更加自然和便捷。这对于有听力或视觉障碍的人群尤为重要，有助于他们更好地理解和享受音视频内容。',
  '4. 情感分析\n描述：大语言模型通过分析文本中的情感倾向和情感表达，帮助企业了解客户反馈和情感状态，从而制定更精准的营销策略或优化客户服务。这种技术还可用于社交媒体监控，实时分析公众对某一主题或事件的情绪和反应。',
  '9. 生物医学研究\n描述：在生物医学领域，大语言模型可用于分析基因组数据、蛋白质相互作用等，加速药物发现和新疗法的研究。例如，通过预测基因变异的功能影响，研究者能够更全面地分析人类基因组的潜在风险和治疗靶点。'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [25]:
store.similarity_search_with_relevance_scores(query="你好", k=2)

[(Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
  0.6922509403579714),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
  0.5035073428496666)]

In [46]:
# 第三个级别
retriever = store.as_retriever(search_type="similarity_score_threshold",
                              search_kwargs={"k": 4, "score_threshold": 0.5})

In [47]:
retriever.invoke(input="你好")

[Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：')]

In [45]:
store.similarity_search_with_relevance_scores(query="你好")

[(Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
  0.6922509403579714),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
  0.5035073428496666),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='综上所述，大语言模型在多个领域都具有广泛的应用前景和巨大的价值潜力。随着技术的不断进步和完善，我们有理由相信大语言模型将在未来的人工智能领域发挥更加重要的作用。'),
  0.4773380274344644),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='4. 情感分析\n描述：大语言模型通过分析文本中的情感倾向和情感表达，帮助企业了解客户反馈和情感状态，从而制定更精准的营销策略或优化客户服务。这种技术还可用于社交媒体监控，实时分析公众对某一主题或事件的情绪和反应。'),
  0.4666226126690547)]